In [ ]:
import nltk
import random
import json
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

In [ ]:
intents ={
  "intents": [
    {
      "tag": "aloe_vera",
      "patterns": [
        "What is Aloe Vera?",
        "What are the benefits of Aloe Vera?",
        "How do I use Aloe Vera?",
        "Are there any side effects of Aloe Vera?",
        "Can Aloe Vera help with skin issues?"
      ],
      "responses": [
        "Aloe Vera is a succulent plant known for its medicinal and cosmetic uses. It has soothing, moisturizing, and healing properties.",
        "The benefits of Aloe Vera include improved skin hydration, reduced inflammation, and support for digestion.",
        "Aloe Vera can be used topically for skin care or consumed in the form of supplements or juices.",
        "Some people may experience skin irritation or allergic reactions. It's best to do a patch test before using it extensively.",
        "Yes, Aloe Vera can help with skin issues such as burns, acne, and dryness due to its healing and moisturizing properties."
      ]
    },
    {
      "tag": "tulsi",
      "patterns": [
        "What is Tulsi?",
        "What are the health benefits of Tulsi?",
        "How can I use Tulsi?",
        "Are there any side effects of Tulsi?",
        "Can Tulsi help with stress?"
      ],
      "responses": [
        "Tulsi, also known as Holy Basil, is an aromatic plant used in traditional medicine for its therapeutic properties.",
        "Tulsi is known for its benefits in reducing stress, boosting immunity, and improving respiratory health.",
        "Tulsi can be consumed as tea, used in cooking, or applied topically in some forms.",
        "Tulsi is generally safe but might interact with certain medications. Consult a healthcare provider if you have any concerns.",
        "Yes, Tulsi is known to have adaptogenic properties that help in reducing stress and balancing the body's response to stress."
      ]
    },
    {
      "tag": "ashwagandha",
      "patterns": [
        "What is Ashwagandha?",
        "What are the benefits of Ashwagandha?",
        "How should I take Ashwagandha?",
        "Are there any side effects of Ashwagandha?",
        "Can Ashwagandha help with sleep?"
      ],
      "responses": [
        "Ashwagandha is an adaptogenic herb used in Ayurvedic medicine to help the body cope with stress and improve overall health.",
        "Benefits of Ashwagandha include stress reduction, improved energy levels, and enhanced cognitive function.",
        "Ashwagandha can be taken as a supplement in various forms, including capsules, powders, or teas.",
        "Some people may experience mild side effects such as digestive upset. It's best to start with a lower dose and consult a healthcare provider.",
        "Yes, Ashwagandha is often used to support better sleep and improve sleep quality due to its calming effects."
      ]
    },
    {
      "tag": "neem",
      "patterns": [
        "What is Neem?",
        "What are the uses of Neem?",
        "How can I use Neem?",
        "Are there any side effects of Neem?",
        "Can Neem help with acne?"
      ],
      "responses": [
        "Neem is a tree known for its medicinal properties, often used in traditional medicine for its antibacterial and anti-inflammatory effects.",
        "Neem can be used for various purposes, including skin care, dental health, and as a natural pesticide.",
        "Neem can be applied topically in the form of creams or oils, or taken internally as supplements.",
        "Neem is generally safe for most people, but it can cause some gastrointestinal issues in high doses. Always consult with a healthcare provider if unsure.",
        "Yes, Neem is known for its antibacterial properties, making it effective in treating acne and other skin infections."
      ]
    },
    {
      "tag": "aloe_vera_uses",
      "patterns": [
        "What are the uses of Aloe Vera?",
        "How can I use Aloe Vera for skin?",
        "Can Aloe Vera be used for digestive issues?",
        "What are some common uses of Aloe Vera?",
        "How is Aloe Vera used in cosmetics?"
      ],
      "responses": [
        "Aloe Vera can be used for soothing burns, moisturizing the skin, treating acne, and improving digestion. It's also used in various cosmetic products for its hydrating properties.",
        "For skin care, Aloe Vera gel can be applied topically to treat burns, cuts, and dry skin.",
        "Aloe Vera juice can aid digestion and support gut health when consumed in moderation.",
        "Common uses include treating minor skin irritations, adding moisture to the skin, and promoting healing of wounds.",
        "In cosmetics, Aloe Vera is used in lotions, creams, and masks for its moisturizing and soothing effects."
      ]
    },
    {
      "tag": "aloe_vera_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Aloe Vera?",
        "What are the disadvantages of Aloe Vera?",
        "Are there any risks of using Aloe Vera?",
        "How does Aloe Vera benefit health?",
        "Can Aloe Vera have side effects?"
      ],
      "responses": [
        "Advantages of Aloe Vera include its ability to soothe skin irritations, improve skin hydration, and support digestive health. It's also known for its anti-inflammatory and antimicrobial properties.",
        "Disadvantages may include potential skin irritation or allergic reactions in some individuals. Overuse of Aloe Vera internally may also lead to digestive issues.",
        "Risks include potential allergic reactions and interactions with certain medications. It's best to use Aloe Vera with caution and consult a healthcare provider if unsure.",
        "Health benefits include reduced inflammation, improved skin healing, and enhanced digestion.",
        "Side effects can include mild irritation or allergic reactions, especially with topical use. Always perform a patch test before extensive use."
      ]
    },
    {
      "tag": "tulsi_uses",
      "patterns": [
        "What are the uses of Tulsi?",
        "How can Tulsi benefit health?",
        "Can Tulsi be used for stress relief?",
        "What are some common uses of Tulsi?",
        "How is Tulsi used in traditional medicine?"
      ],
      "responses": [
        "Tulsi, or Holy Basil, is used for stress relief, boosting immunity, and improving respiratory health. It's commonly consumed as tea, used in cooking, or applied topically.",
        "For stress relief, Tulsi tea can be consumed daily. Tulsi leaves can also be used in cooking for their health benefits.",
        "Common uses include making Tulsi tea for immune support, adding Tulsi leaves to dishes, or using Tulsi oil for topical applications.",
        "In traditional medicine, Tulsi is used to balance the body's systems, support respiratory function, and reduce stress."
      ]
    },
    {
      "tag": "tulsi_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Tulsi?",
        "What are the disadvantages of Tulsi?",
        "Are there any risks of using Tulsi?",
        "How does Tulsi benefit health?",
        "Can Tulsi have side effects?"
      ],
      "responses": [
        "Advantages of Tulsi include its ability to reduce stress, boost immunity, and support respiratory health. It also has anti-inflammatory and antioxidant properties.",
        "Disadvantages may include potential interactions with medications and mild gastrointestinal issues. It's best to consult with a healthcare provider before extensive use.",
        "Risks include potential interactions with blood-thinning medications and digestive upset in some individuals.",
        "Health benefits include improved stress management, enhanced immunity, and support for respiratory health.",
        "Side effects are generally rare but may include mild digestive discomfort or interactions with certain medications."
      ]
    },
    {
      "tag": "ashwagandha_uses",
      "patterns": [
        "What are the uses of Ashwagandha?",
        "How can Ashwagandha benefit health?",
        "Can Ashwagandha be used for anxiety?",
        "What are some common uses of Ashwagandha?",
        "How is Ashwagandha used in supplements?"
      ],
      "responses": [
        "Ashwagandha is used for reducing stress, improving energy levels, supporting cognitive function, and enhancing overall vitality. It’s commonly taken as a supplement in capsules, powders, or teas.",
        "For anxiety, Ashwagandha can be taken as a supplement to help manage stress and promote relaxation.",
        "Common uses include taking Ashwagandha supplements for stress relief, improved sleep, and enhanced cognitive function.",
        "In supplements, Ashwagandha is available in various forms such as capsules, powders, and extracts."
      ]
    },
    {
      "tag": "ashwagandha_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Ashwagandha?",
        "What are the disadvantages of Ashwagandha?",
        "Are there any risks of using Ashwagandha?",
        "How does Ashwagandha benefit health?",
        "Can Ashwagandha have side effects?"
      ],
      "responses": [
        "Advantages of Ashwagandha include its ability to reduce stress, enhance cognitive function, and improve overall vitality. It's known for its adaptogenic properties and support for better sleep.",
        "Disadvantages may include potential gastrointestinal issues, interactions with medications, and mild drowsiness in some individuals.",
        "Risks include potential interactions with sedatives and thyroid medications. It's best to use Ashwagandha under the guidance of a healthcare provider.",
        "Health benefits include reduced stress, improved sleep quality, and enhanced cognitive function.",
        "Side effects can include digestive upset, drowsiness, or interactions with certain medications. Consult a healthcare provider for personalized advice."
      ]
    },
    {
      "tag": "neem_uses",
      "patterns": [
        "What are the uses of Neem?",
        "How can Neem benefit health?",
        "Can Neem be used for skin care?",
        "What are some common uses of Neem?",
        "How is Neem used in traditional medicine?"
      ],
      "responses": [
        "Neem is used for its antibacterial, antifungal, and anti-inflammatory properties. Common uses include treating skin conditions, improving oral health, and acting as a natural pesticide.",
        "For skin care, Neem can be used topically as an oil or in creams to treat acne and other skin issues.",
        "Common uses include using Neem oil for skin and hair care, Neem sticks for dental health, and Neem extracts as natural pesticides.",
        "In traditional medicine, Neem is used to treat a variety of conditions including skin infections, digestive issues, and as a general health tonic."
      ]
    },
    {
      "tag": "neem_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Neem?",
        "What are the disadvantages of Neem?",
        "Are there any risks of using Neem?",
        "How does Neem benefit health?",
        "Can Neem have side effects?"
      ],
      "responses": [
        "Advantages of Neem include its antibacterial, antifungal, and anti-inflammatory properties, making it effective for skin care, dental health, and as a natural pesticide.",
        "Disadvantages may include potential skin irritation or allergic reactions in some individuals. Neem can also have a strong odor which some people find unpleasant.",
        "Risks include potential allergic reactions or gastrointestinal issues if consumed in large amounts. It's best to consult with a healthcare provider if using Neem extensively.",
        "Health benefits include improved skin health, enhanced oral hygiene, and natural pest control.",
        "Side effects can include mild skin irritation, digestive upset, or allergic reactions in some people. Use Neem products cautiously and consult a healthcare provider if needed."
      ]
    }
  ]
}

In [ ]:
# Function to perform synonym replacement
def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i+1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
text_data = []
labels = []
stopwords = set(nltk.corpus.stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


limit_per_tag = 40

for intent in intents['intents']:
    augmented_sentences_per_tag = 0
    for example in intent['patterns']:
        tokens = nltk.word_tokenize(example.lower())
        filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords and token.isalpha()]
        if filtered_tokens:
            text_data.append(' '.join(filtered_tokens))
            labels.append(intent['tag'])

            augmented_sentences = synonym_replacement(filtered_tokens, limit_per_tag - augmented_sentences_per_tag)
            for augmented_sentence in augmented_sentences:
                text_data.append(augmented_sentence)
                labels.append(intent['tag'])
                augmented_sentences_per_tag += 1
                if augmented_sentences_per_tag >= limit_per_tag:
                    break

print(len(text_data))
print(len(labels))


540
540


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = labels

In [ ]:
def find_best_model(X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=100)


    models = [
        ('Logistic Regression', LogisticRegression(), {
            'penalty': ['l2'],
            'C': [0.1, 1.0, 10.0],
            'solver': ['liblinear'],
            'max_iter': [100, 1000, 10000]
        }),
        ('Multinomial Naive Bayes', MultinomialNB(), {'alpha': [0.1, 0.5, 1.0]}),
        ('Linear SVC', LinearSVC(), {
            'penalty': ['l2'],
            'loss': ['hinge', 'squared_hinge'],
            'C': [0.1, 1, 10],
            'max_iter': [100, 1000, 10000]
        }),
        ('Decision Tree', DecisionTreeClassifier(), {
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy']
        }),
        ('Random Forest', RandomForestClassifier(), {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        })
    ]

    for name, model, param_grid in models:
        grid = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        print(f'{name}: {score:.4f} (best parameters: {grid.best_params_})')

    best_model = max(models, key=lambda x: GridSearchCV(x[1], x[2], cv=3, n_jobs=-1).fit(X_train, y_train).score(X_test, y_test))
    print(f'\nBest model: {best_model[0]}')

    # Fit the best model to the full training data
    best_model[1].fit(X, y)

    return best_model[1]

In [ ]:
best_model = find_best_model(X, y)

Logistic Regression: 0.7870 (best parameters: {'C': 10.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'})
Multinomial Naive Bayes: 0.7130 (best parameters: {'alpha': 0.5})
Linear SVC: 0.7685 (best parameters: {'C': 1, 'loss': 'squared_hinge', 'max_iter': 100, 'penalty': 'l2'})
Decision Tree: 0.8056 (best parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5})
Random Forest: 0.7778 (best parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100})

Best model: Decision Tree


In [ ]:
def chatbot_response(user_input):
    input_text = vectorizer.transform([user_input])
    predicted_intent = best_model.predict(input_text)[0]

    for intent in intents['intents']:
        if intent['tag'] == predicted_intent:
            response = random.choice(intent['responses'])
            break

    return response

In [ ]:
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = chatbot_response(user_input)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
> What is Tulsi?
For stress relief, Tulsi tea can be consumed daily. Tulsi leaves can also be used in cooking for their health benefits.
> What are the advantages of Tulsi?
Tulsi, or Holy Basil, is used for stress relief, boosting immunity, and improving respiratory health. It's commonly consumed as tea, used in cooking, or applied topically.
> What are the benefits of Aloe Vera?
Aloe Vera can be used for soothing burns, moisturizing the skin, treating acne, and improving digestion. It's also used in various cosmetic products for its hydrating properties.
> what is aloe vera
Aloe Vera juice can aid digestion and support gut health when consumed in moderation.
> quit


In [ ]:
import os
import pickle


if not os.path.exists('model'):
    os.makedirs('model')

if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Save the trained model
with open('model/chatbot_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save the vectorizer
with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Save the intents to the "dataset" folder
with open('dataset/intents1.json', 'w') as f:
    json.dump(intents, f)